In [2]:
from pathlib import Path
import subprocess as sb
from subprocess import CalledProcessError
import shutil
from sys import exit, argv
from datetime import datetime
from re import match


class CX1job(object): 
    job_id = None
    jobscript = Path.home() / 'FKMC/batchscripts/CX1_jobscript.sh'
    running = False
    
    def __init__(self, python_script, job_name, job_folder_name, array_indices): 
        self.python_script = python_script
        self.job_name = job_name
        self.job_folder_name = job_folder_name
        self.array_indices = array_indices
        self.submit_dir = Path('/rds/general/user/tch14/home/HPC_data') / job_folder_name
      
    def submit(self, held = False):
        #http://docs.adaptivecomputing.com/torque/4-0-2/Content/topics/commands/qsub.htm
        start, stop = self.array_indices
        indices = f'{start}-{stop}'
    
        qsub_args = ['qsub', 
                     '-h' if held else '', 
                     '-v', f'PYTHON_SCRIPT={self.python_script.name}, SUBMIT_DIR={self.submit_dir}',
                     '-N', f'{self.job_name}',
                     '-J', f'{indices}',
                     '-lselect=1:ncpus=1:mem=4gb:avx=true',
                     #'-lwalltime=24:00:00',
                     '-lwalltime=00:00:60',
                     '-o', str(self.submit_dir / 'logs'),
                     '-e', str(self.submit_dir / 'logs'),
                   str(self.jobscript)]
        print(' '.join(qsub_args))
        
        try:
            #jobid has the form 1649928[].pbs
            jobstring = sb.check_output(qsub_args, encoding = 'utf8', stderr=sb.PIPE)
            self.job_id = match(r'(\d+)\[\]\.pbs', jobstring).groups()[0]
            
        except CalledProcessError as e:
            print(e.output, e.stderr, e.returncode)
            raise e
        
        print(f'Job created with id {self.job_id}')
        return self

    def cancel(self):
        if self.job_id == None: 
            print("Can't cancel, no job_id")
            return
        print(f'Cancelling job with id {self.job_id}')
        try:
            sb.check_output(['qdel', self.job_id + '[]'], encoding = 'utf8', stderr=sb.PIPE)
        except CalledProcessError as e:
            print(e.output, e.stderr, e.returncode)
            raise e
    
    def release(self):
        if self.job_id == None: 
            print("Can't release, no job_id")
            return
        print(f'Releasing job with id {self.job_id}')
        try:
            output = sb.check_output(['qrls', self.job_id + '[]'], encoding = 'utf8', stderr=sb.PIPE)
            self.running = True
        except CalledProcessError as e:
            print(e.output, e.stderr, e.returncode)
            raise e
            
    def status(self):
        try:
            output = sb.check_output(['qstat', self.job_id], encoding = 'utf8', stderr=sb.PIPE)
        except CalledProcessError as e:
            #print(e.output, e.stderr, e.returncode)
            return 'Job finished'
        return output
        

In [100]:
### resolve paths to the scripts
ipynb_script = Path('test_batchscript.ipynb').resolve() #resolve give the absolute path
py_script = ipynb_script.parent / (ipynb_script.stem + '.py')
job_name = ipynb_script.stem
job_folder_name = f'{job_name}_{time.time():.0f}'
indices = (0,5)

### Regenerate the py from the ipynb based on timestamps
if ipynb_script.stat().st_mtime > py_script.stat().st_mtime:
    print('Regenerating py script from ipynb')
    sb.check_output(['jupyter', 'nbconvert', '--to', 'script', ipynb_script])   

### make the job which gives access to some platform specific info like paths and such
job = CX1job(py_script, job_name, job_folder_name, indices)

### Make the file where the code and data will be saved
code_dir = job.submit_dir / 'code'
data_dir = job.submit_dir / 'data'
logs_dir = job.submit_dir / 'logs'
for d in [code_dir, data_dir, logs_dir]: d.mkdir(parents=True, exist_ok=True)

### copy the code over
shutil.copy(str(py_script), code_dir)
shutil.copy(str(ipynb_script), code_dir)

job.submit(held = True)
time.sleep(0.1)
job.release()

qsub -h -v PYTHON_SCRIPT=test_batchscript.py, SUBMIT_DIR=/rds/general/user/tch14/home/HPC_data/test_batchscript_1593789594 -N test_batchscript -J 0-5 -lselect=1:ncpus=1:mem=4gb:avx=true -lwalltime=00:00:60 -o /rds/general/user/tch14/home/HPC_data/test_batchscript_1593789594/logs -e /rds/general/user/tch14/home/HPC_data/test_batchscript_1593789594/logs /rds/general/user/tch14/home/FKMC/batchscripts/CX1_jobscript.sh
Job created with id 1813741
Releasing job with id 1813741


In [87]:
job.status()

'Job finished'

In [63]:
job.cancel()

Cancelling job with id 1811744


In [92]:
!qstat

/bin/sh: module: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `BASH_FUNC_module'
   Job ID           Class            Job Name        Status     Comment   
-------------- --------------- -------------------- -------- -------------
1811230        Jupyter         jupyterhub           Running  finishing today at 17:20


In [85]:
!qdel 1811899[]

/bin/sh: module: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `BASH_FUNC_module'


In [82]:
a = ['mv', '/rds/general/user/tch14/home/HPC_data/logs/*.pbs.*', '/rds/general/user/tch14/home/HPC_data/old_logs']
b = ' '.join(a)
b

'mv /rds/general/user/tch14/home/HPC_data/logs/*.pbs.* /rds/general/user/tch14/home/HPC_data/old_logs'

In [91]:
time.time()

1593777163.0801446

In [2]:
a = [1,2,3]
a.insert(-1, 4)
a

[1, 2, 4, 3]

In [16]:
qstat = sb.check_output(['qstat'], encoding = 'utf8')  
#qstat = sb.check_output(['qstat'], encoding = 'utf8')   
qstat = [row.split() for row in qstat.split('\n')][2:-1]
' '.join([r[0] if len(r)>0 else r for r in qstat])

'1882836[].pbs 1882837[].pbs 1882838[].pbs 1882839[].pbs 1882840[].pbs 1882841[].pbs 1882842[].pbs 1885840.pbs'

In [27]:
%load_ext autoreload
%autoreload 2
%run ~/FKMC/notebooks/notebook_preamble.py
%matplotlib inline
np.seterr(under = 'ignore')
from time import time
from munch import munchify

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Imported pathlib::Path, os, munch::Munch, pickle, logging, itertools::count, matplotlib::colors
Names are pyplot = plt, matplotlib = mpl, numpy = np


In [42]:
submit_dir = Path('/home/tch14/HPC_data/Tsweep_U5_J5_test')
filename = '35_1.npz'
d = munchify(np.load(submit_dir / 'data' / filename, allow_pickle = True))
logs = d['logs'][()]
logs

array([<FKMC.montecarlo.Eigenspectrum_IPR_all object at 0x1554614a6cd0>,
      dtype=object)

In [43]:
[l.N_sites if l else None for l in logs]

[70, 80, 90, 110, 130, 150, 180, 210, 250, 300]

In [39]:
os.uname()

posix.uname_result(sysname='Linux', nodename='cx1-101-17-3.cx1.hpc.ic.ac.uk', release='4.18.0-193.6.3.el8_2.x86_64', version='#1 SMP Wed Jun 10 11:09:32 UTC 2020', machine='x86_64')